In [1]:
import xml.etree.ElementTree as et
import pandas as pd
from IPython.core import display as ICD
from pandas import json_normalize
import pandas as pd
import tqdm as tqdm
import os

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

Parsing opta files

In [2]:
#@title
types_dict={1:"Pass",2:"Offside Pass",3:"Take On",4:"Foul",5:"Out",6:"Corner Awarded",7:"Tackle",8:"Interception",9:"Turnover",10:"Save",11:"Claim",12:"Clearance",13:"Miss",14:"Post",15:"Attempt Saved",16:"Goal",17:"Card",18:"Player off",19:"Player on",20:"Player retired",21:"Player returns",22:"Player becomes goalkeeper",23:"Goalkeeper becomes player",24:"Condition change",25:"Official change",27:"Start delay",28:"End delay",30:"End",31:"Picked an orange",32:"Start",34:"Team set up",35:"Player changed position",36:"Player changed Jersey number",37:"Collection End",38:"Temp_Goal",39:"Temp_Attempt",40:"Formation change",41:"Punch",42:"Good Skill",43:"Deleted event",44:"Aerial",45:"Challenge",47:"Rescinded card",49:"Ball recovery",50:"Dispossessed",51:"Error",52:"Keeper pick-up",53:"Cross not claimed",54:"Smother",55:"Offside provoked",56:"Shield ball opp",57:"Foul throw-in",58:"Penalty faced",59:"Keeper Sweeper",60:"Chance missed",61:"Ball touch",63:"Temp_Save",64:"Resume",65:"Contentious referee decision",67:"50/50",68:"Referee drop ball",69:"Failed To Block",72:"Caught offside",73:"Other Ball Contact",74:"Blocked pass"};
types = pd.DataFrame.from_dict(types_dict,orient='index').reset_index()
types.columns=["type_id","event_name"]

qualifiers_dict={1:"Long ball",2:"Cross",3:"Head pass",4:"Through ball",5:"Free kick taken",6:"Corner taken",7:"Players caught offside",8:"Goal disallowed",9:"Penalty",10:"Hand",11:"6-seconds violation",12:"Dangerous play",13:"Foul",14:"Last line",15:"Head",16:"Small box-centre",17:"Box-centre",18:"Out of box-centre",19:"35+ centre",20:"Right footed",21:"Other body part",22:"Regular play",23:"Fast break",24:"Set piece",25:"From corner",26:"Free kick",28:"Own goal",29:"Assisted",30:"Involved",31:"Yellow Card",32:"Second yellow",33:"Red card",34:"Referee abuse",35:"Argument",36:"Fight",37:"Time wasting",38:"Excessive celebration",39:"Crowd interaction",40:"Other reason",41:"Injury",42:"Tactical",44:"Player position",49:"Attendance figure",50:"Official position",51:"Official ID",53:"Injured player id",54:"End cause",55:"Related event ID",56:"Zone",57:"End type",59:"Jersey number",60:"Small box-right",61:"Small box-left",62:"Box-deep right",63:"Box-right",64:"Box-left",65:"Box-deep left",66:"Out of box-deep right",67:"Out of box-right",68:"Out of box-left",69:"Out of box-deep left",70:"35+ right",71:"35+ left",72:"Left footed",73:"Left",74:"High",75:"Right",76:"Low left",77:"High left",78:"Low centre",79:"High centre",80:"Low right",81:"High right",82:"Blocked",83:"Close left",84:"Close right",85:"Close high",86:"Close left and high",87:"Close right and high",88:"High claim",89:"1 on 1",90:"Deflected save",91:"Dive and deflect",92:"Catch",93:"Dive and catch",94:"Def block",95:"Back pass",96:"Corner situation",97:"Direct free",100:"Six yard blocked",101:"Saved off line",102:"Goal mouth y co-ordinate",103:"Goal mouth z co-ordinate",106:"Attacking Pass",107:"Throw-in",108:"Volley",109:"Overhead",110:"Half Volley",111:"Diving Header",112:"Scramble",113:"Strong",114:"Weak",115:"Rising",116:"Dipping",117:"Lob",118:"One Bounce",119:"Few Bounces",120:"Swerve Left",121:"Swerve Right",122:"Swerve Moving",123:"Keeper Throw",124:"Goal Kick",127:"Direction of play",128:"Punch",130:"Team formation",131:"Team player formation",132:"Dive",133:"Deflection",134:"Far Wide Left",135:"Far Wide Right",136:"Keeper Touched",137:"Keeper Saved",138:"Hit Woodwork",139:"Own Player",140:"Pass End X",141:"Pass End Y",144:"Deleted event type",145:"Formation slot",146:"Blocked x co-ordinate",147:"Blocked y co-ordinate",153:"Not past goal line",154:"Intentional assist",155:"Chipped",156:"Lay-off",157:"Launch",158:"Persistent infringement",159:"Foul and abusive language",160:"Throw-in set piece",161:"Encroachment",162:"Leaving field",163:"Entering field",164:"Spitting",165:"Professional foul",166:"Handling on the line",167:"Out of play",168:"Flick-on",169:"Leading to attempt",170:"Leading to goal",171:"Rescinded card",172:"No impact on timing",173:"Parried safe",174:"Parried danger",175:"Fingertip",176:"Caught",177:"Collected",178:"Standing",179:"Diving",180:"Stooping",181:"Reaching",182:"Hands",183:"Feet",184:"Dissent",185:"Blocked cross",186:"Scored",187:"Saved",188:"Missed",189:"Player not visible",190:"From shot off target",191:"Off the ball foul",192:"Block by hand",194:"Captain",195:"Pull Back",196:"Switch of play",197:"Team kit",198:"GK hoof",199:"Gk kick from hands",200:"Referee stop",201:"Referee delay",202:"Weather problem",203:"Crowd trouble",204:"Fire",205:"Object thrown on pitch",206:"Spectator on pitch",207:"Awaiting officials decision",208:"Referee Injury",209:"Game end",210:"Assist",211:"Overrun",212:"Length",213:"Angle",214:"Big Chance",215:"Individual Play",216:"2nd related event ID",217:"2nd assisted",218:"2nd assist",219:"Players on both posts",220:"Player on near post",221:"Player on far post",222:"No players on posts",223:"In-swinger",224:"Out-swinger",225:"Straight",226:"Suspended",227:"Resume",228:"Own shot blocked",229:"Post-match complete",230:"GK X Coordinate",231:"GK Y Coordinate",232:"Unchallenged"};
qualifiers = pd.DataFrame.from_dict(qualifiers_dict,orient='index').reset_index()
qualifiers.columns = ["qualifier_id","description"]

qualifiers_dict2 = {str(key): str(value) for key, value in qualifiers_dict.items()}


def parsef24(F24file): 
  tree = et.ElementTree(file = F24file)
  games = tree.getroot()
  gameinfo = games.findall('Game')
  gameinfo = gameinfo[0]
  game_id = gameinfo.get('id')
  home_team_id = gameinfo.get('home_team_id')
  home_team_name = gameinfo.get('home_team_name')
  away_team_id = gameinfo.get('away_team_id')
  away_team_name = gameinfo.get('away_team_name')
  competition_id = gameinfo.get('competition_id')
  competition_name = gameinfo.get('competition_name')
  season_id = gameinfo.get('season_id')

  global match_events
  match_events = pd.DataFrame()
  for game in games:
      for event in game:
          evento = json_normalize(event.attrib)
          mylist = []
          for q in event:
            mylist.append(q.attrib)
          evento["qualifiers"] = ["a"]
          evento.at[0,"qualifiers"] = mylist
          match_events = match_events.append(evento)
  match_events[["id","event_id","type_id","period_id","min","sec"]] = match_events[["id","event_id","type_id","period_id","min","sec"]].astype(int)
  match_events[["y","x"]] = match_events[["y","x"]].astype(float)
  match_events = pd.merge(match_events,types, on="type_id", how = "left")
  match_events = match_events[ ['id',"event_id","type_id", "event_name" ]+ [ col for col in match_events.columns if col not in ['id',"event_id","type_id", "event_name" ] ] ]
  match_events['competition_id'] = competition_id
  match_events['game_id'] = game_id
  match_events['home_team_id'] = home_team_id
  match_events['home_team_name'] = home_team_name
  match_events['away_team_id'] = away_team_id
  match_events['away_team_name'] = away_team_name
  match_events['competition_id'] = competition_id
  match_events['competition_name'] = competition_name
  match_events['season_id'] = season_id
  match_events['competition_id'] = competition_id
  #ICD.display(match_events.head(10))

def search_qualifier(nome_df, qualifier_id, lista_destino): 
  for i in range(len(nome_df)):
    for item in nome_df.loc[i]["qualifiers"]: 
      if item["qualifier_id"] == qualifier_id: 
        lista_destino.append(nome_df.loc[i]["id"])
  return ICD.display(nome_df[nome_df["id"].isin(lista_destino)])

def explode_event(nome_df,id_evento,mytresh):
  global exploded_df
  exploded_df = pd.DataFrame()
  nome_df=nome_df[nome_df["type_id"]==id_evento]
  allqualifiers = pd.DataFrame()
  for i in range(len(nome_df)):
    qualifiers_detailed = json_normalize(nome_df.qualifiers.iloc[i]).T.reset_index().drop("index",axis=1).fillna("sim")
    qualifiers_detailed.columns = list(qualifiers_detailed.iloc[1])
    qualifiers_detailed = qualifiers_detailed.drop(labels=[0,1], axis=0)
    qualifiers_detailed.rename(columns=qualifiers_dict2,inplace=True)
    qualifiers_detailed["id"] = nome_df.id.iloc[i]
    allqualifiers = allqualifiers.append(qualifiers_detailed)
  exploded_df = exploded_df.append(pd.merge(nome_df.drop("qualifiers", axis=1), allqualifiers.reindex(sorted(allqualifiers.columns), axis=1).reset_index()\
                                            .drop("index", axis=1).dropna(thresh=len(allqualifiers)*mytresh, axis=1), on="id", how="outer")).fillna("-")


In [ ]:
# Parsing one file:
parsef24("f24-99-2020-2133473-eventdetails.xml")

In [ ]:
match_events.info()

In [ ]:
explode_event(match_events,1,0.15)

In [ ]:
exploded_df[["Pass End X","Pass End Y"]] = exploded_df[["Pass End X","Pass End Y"]].astype(float)
# converting passing locations to floats

In [ ]:
exploded_df.info() # need to make the previous conversion to work properly

Let's make the previous process for all the Portuguese league 20/21 files and the half-space passing calculations as well.

I'm analysing the passing for offensive half-spaces because because I believe that a lot of danger and opportunities can be created through these zones, so getting the ball there is crucial.

I define these zones (left and right) and calculate the number and accuracy of passes for these zones, grouped by positions, players and teams in the league. 

In [3]:
# create dataframe
allmatches = pd.DataFrame()

# loop for parsing and make calculations
for file in tqdm.tqdm(os.listdir("/content")):
  if file.endswith(".xml"):
    parsef24("/content/" + file)

    explode_event(match_events,1,0.15)
    # passing coordinates conversion objects to floats

    exploded_df[["Pass End X","Pass End Y"]] = exploded_df[["Pass End X","Pass End Y"]].astype(float)
    
    # right and left offensive half-spaces definition zones
    halfd = exploded_df[(exploded_df["Pass End X"].between(70,83)) & (exploded_df["Pass End Y"].between(21.1,36.8))]
    halfe = exploded_df[(exploded_df["Pass End X"].between(70,83)) & (exploded_df["Pass End Y"].between(63.2,78.9))]
    
    
    passes_completedd = halfd[halfd["outcome"] == "1"].groupby(["player_id"]).count()[["id"]].reset_index()
    passes_completedd.columns = ["player_id", "Passes completed right"]
    passes_uncompletedd = halfd[halfd["outcome"] == "0"].groupby(["player_id"]).count()[["id"]].reset_index()
    passes_uncompletedd.columns=["player_id", "Passes uncompleted right"]
    passes_completede = halfe[halfe["outcome"] == "1"].groupby(["player_id"]).count()[["id"]].reset_index()
    passes_completede.columns = ["player_id", "Passes completed left"]
    passes_uncompletede = halfe[halfe["outcome"] == "0"].groupby(["player_id"]).count()[["id"]].reset_index()
    passes_uncompletede.columns = ["player_id", "Passes uncompleted left"]
    
    passesd = pd.merge(passes_completedd,passes_uncompletedd, on="player_id", how = "outer")
    passese = pd.merge(passes_completede,passes_uncompletede, on="player_id", how = "outer")
    passesf = pd.merge(passese,passesd,on="player_id",how ="outer").fillna(0)
    passesf = passesf.astype(int)
    
    # total passes  in offensive half-spaces
    passesf["Total passes"] = passesf["Passes completed left"]+passesf["Passes completed right"]+passesf["Passes uncompleted left"]+passesf["Passes uncompleted right"]
    # total completed passes in offensive half-spaces
    passesf["Total passes completed"] = passesf["Passes completed left"]+passesf["Passes completed right"]
    # % completed passes offensive half-spaces
    passesf["% Total passes completed"] = (passesf["Passes completed left"]+passesf["Passes completed right"]) * 100 / passesf["Total passes"]
    # total passes in left offensive half-space
    passesf["Total passes left"] = passesf["Passes completed left"]+passesf["Passes uncompleted left"]
    # % completed passes in left offensive half-space
    passesf["% Total passes completed left"] = passesf["Passes completed left"] * 100 / passesf["Total passes left"]
    # total passes in right offensive half-space
    passesf["Total passes right"] = passesf["Passes completed right"]+passesf["Passes uncompleted right"]
    # % completed passes in right offensive half-space
    passesf["% Total passes completed right"] = passesf["Passes completed right"] * 100 / passesf["Total passes right"]
    passesf = passesf.fillna(0)

    allmatches = allmatches.append(passesf)


100%|██████████| 303/303 [53:34<00:00, 10.61s/it]


In [4]:
allmatches.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5293 entries, 0 to 17
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   player_id                       5293 non-null   int64  
 1   Passes completed left           5293 non-null   int64  
 2   Passes uncompleted left         5293 non-null   int64  
 3   Passes completed right          5293 non-null   int64  
 4   Passes uncompleted right        5293 non-null   int64  
 5   Total passes                    5293 non-null   int64  
 6   Total passes completed          5293 non-null   int64  
 7   % Total passes completed        5293 non-null   float64
 8   Total passes left               5293 non-null   int64  
 9   % Total passes completed left   5293 non-null   float64
 10  Total passes right              5293 non-null   int64  
 11  % Total passes completed right  5293 non-null   float64
dtypes: float64(3), int64(9)
memory usage

In [5]:
all = allmatches.copy

In [6]:
planteispt = pd.read_excel("opta_planteis_portugal.xlsx")  

In [7]:
allmatches["player_id"] = allmatches["player_id"].astype(int)
# merging parse dataframe with player information dataframe
allmatches = pd.merge(allmatches,planteispt[["player_id","player_name","position","team_name"]], on ="player_id", how = "left")

In [8]:
allmatches=allmatches.fillna(0)
allmatches.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5428 entries, 0 to 5427
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   player_id                       5428 non-null   int64  
 1   Passes completed left           5428 non-null   int64  
 2   Passes uncompleted left         5428 non-null   int64  
 3   Passes completed right          5428 non-null   int64  
 4   Passes uncompleted right        5428 non-null   int64  
 5   Total passes                    5428 non-null   int64  
 6   Total passes completed          5428 non-null   int64  
 7   % Total passes completed        5428 non-null   float64
 8   Total passes left               5428 non-null   int64  
 9   % Total passes completed left   5428 non-null   float64
 10  Total passes right              5428 non-null   int64  
 11  % Total passes completed right  5428 non-null   float64
 12  player_name                     54

In [9]:
allmatches["position"]=allmatches["position"].astype(str)

# % passing accuracy per zonas and player position in the league

accuracy_position = allmatches.groupby("position").sum()

# recalculate % completed passes
accuracy_position["% Total passes completed"] = (accuracy_position["Passes completed left"]+accuracy_position["Passes completed right"]) * 100 / accuracy_position["Total passes"]
accuracy_position["% Total passes completed left"] = (accuracy_position["Passes completed left"]) * 100 / accuracy_position["Total passes left"]
accuracy_position["% Total passes completed right"] = (accuracy_position["Passes completed right"]) * 100 / accuracy_position["Total passes right"]
accuracy_position.sort_values("% Total passes completed", ascending=False).fillna(0).reset_index().drop(2).drop(["player_id"], axis = 1)

,position,Passes completed left,Passes uncompleted left,Passes completed right,Passes uncompleted right,Total passes,Total passes completed,% Total passes completed,Total passes left,% Total passes completed left,Total passes right,% Total passes completed right
0,Midfielder,1324,725,1475,892,4416,2799,63.383152,2049,64.616886,2367,62.315167
1,Forward,713,466,657,458,2294,1370,59.721011,1179,60.474979,1115,58.923767
3,Defender,800,701,883,728,3112,1683,54.080977,1501,53.297801,1611,54.810677
4,Goalkeeper,56,184,73,223,536,129,24.067164,240,23.333333,296,24.662162


In [10]:
allmatches["team_name"]=allmatches["team_name"].astype(str)
# % passing accuracy per zones and team in the league

accuracy_teams = allmatches.groupby("team_name").sum()
# recalculate % completed passes
accuracy_teams["% Total passes completed"] = ((accuracy_teams["Passes completed left"]+accuracy_teams["Passes completed right"]) * 100 / accuracy_teams["Total passes"]).sort_values()
accuracy_teams["% Total passes completed left"] = (accuracy_teams["Passes completed left"]) * 100 / accuracy_teams["Total passes left"]
accuracy_teams["% Total passes completed right"] = (accuracy_teams["Passes completed right"]) * 100 / accuracy_teams["Total passes right"]
accuracy_teams.sort_values("% Total passes completed", ascending=False).fillna(0).reset_index().drop(7).drop(["player_id"], axis = 1)

,team_name,Passes completed left,Passes uncompleted left,Passes completed right,Passes uncompleted right,Total passes,Total passes completed,% Total passes completed,Total passes left,% Total passes completed left,Total passes right,% Total passes completed right
0,Benfica,336,113,397,151,997,733,73.520562,449,74.832962,548,72.445255
1,FC Porto,221,133,287,165,806,508,63.027295,354,62.429379,452,63.495575
2,Sporting Braga,233,132,219,138,722,452,62.603878,365,63.835616,357,61.344538
3,Sporting CP,168,116,193,133,610,361,59.180328,284,59.154930,326,59.202454
4,Gil Vicente,159,114,177,119,569,336,59.050967,273,58.241758,296,59.797297
5,Boavista,163,134,132,77,506,295,58.300395,297,54.882155,209,63.157895
6,Vitória Guimarães,162,96,186,164,608,348,57.236842,258,62.790698,350,53.142857
8,Rio Ave,167,121,129,105,522,296,56.704981,288,57.986111,234,55.128205
9,FC Famalicão,142,108,132,106,488,274,56.147541,250,56.800000,238,55.462185
10,Tondela,161,125,157,124,567,318,56.084656,286,56.293706,281,55.871886


In [11]:
#  passing accuracy per zones by player in the league

accuracy_player = allmatches.groupby("player_name").sum().sort_values("Total passes completed", ascending=False).fillna(0).drop(["player_id"], axis = 1).reset_index()
# recalculate % completed passes
accuracy_player["% Total passes completed"] = (accuracy_player["Passes completed left"]+accuracy_player["Passes completed right"]) * 100 / accuracy_player["Total passes"]
accuracy_player["% Total passes completed left"] = (accuracy_player["Passes completed left"]) * 100 / accuracy_player["Total passes left"]
accuracy_player["% Total passes completed right"] = (accuracy_player["Passes completed right"]) * 100 / accuracy_player["Total passes right"]
accuracy_player.fillna(0).drop(1)

,player_name,Passes completed left,Passes uncompleted left,Passes completed right,Passes uncompleted right,Total passes,Total passes completed,% Total passes completed,Total passes left,% Total passes completed left,Total passes right,% Total passes completed right
0,Nuno Santos,58,24,30,16,128,88,68.750000,82,70.731707,46,65.217391
2,Rafa,20,6,60,15,101,80,79.207921,26,76.923077,75,80.000000
3,Adel Taarabt,29,5,41,23,98,70,71.428571,34,85.294118,64,64.062500
4,Pizzi,29,8,39,9,85,68,80.000000,37,78.378378,48,81.250000
5,Paulinho,35,14,32,15,96,67,69.791667,49,71.428571,47,68.085106
6,Ricardo Esgaio,0,2,61,22,85,61,71.764706,2,0.000000,83,73.493976
7,Otávio,33,21,28,18,100,61,61.000000,54,61.111111,46,60.869565
8,Éverton,45,10,12,4,71,57,80.281690,55,81.818182,16,75.000000
9,Wenderson Galeno,48,16,9,5,78,57,73.076923,64,75.000000,14,64.285714
10,Jesús Corona,12,8,43,23,86,55,63.953488,20,60.000000,66,65.151515


In [14]:
# % passing accuracy per zones and position by Benfica in the league
accuracy_benfica_position = allmatches[allmatches["team_name"]=="Benfica"].groupby("position").sum().sort_values("Total passes completed", ascending=False).fillna(0)
# recalculate % completed passes
accuracy_benfica_position["% Total passes completed"] = (accuracy_benfica_position["Passes completed left"]+accuracy_benfica_position["Passes completed right"]) * 100 / accuracy_benfica_position["Total passes"]
accuracy_benfica_position["% Total passes completed left"] = (accuracy_benfica_position["Passes completed left"]) * 100 / accuracy_benfica_position["Total passes left"]
accuracy_benfica_position["% Total passes completed right"] = (accuracy_benfica_position["Passes completed right"]) * 100 / accuracy_benfica_position["Total passes right"]
accuracy_benfica_position

,player_id,Passes completed left,Passes uncompleted left,Passes completed right,Passes uncompleted right,Total passes,Total passes completed,% Total passes completed,Total passes left,% Total passes completed left,Total passes right,% Total passes completed right
position,,,,,,,,,,,,
Midfielder,26348705,155,43,243,82,523,398,76.099426,198,78.282828,325,74.769231
Defender,21407745,86,35,82,34,237,168,70.886076,121,71.074380,116,70.689655
Forward,18749043,94,32,71,33,230,165,71.739130,126,74.603175,104,68.269231
Goalkeeper,899707,1,3,1,2,7,2,28.571429,4,25.000000,3,33.333333


In [12]:
# % passing accuracy per zones and position by FCP in the league
accuracy_fcp_position = allmatches[allmatches["team_name"]=="FC Porto"].groupby("position").sum().sort_values("Total passes completed", ascending=False).fillna(0)
# recalculate % completed passes
accuracy_fcp_position["% Total passes completed"] = (accuracy_fcp_position["Passes completed left"]+accuracy_fcp_position["Passes completed right"]) * 100 / accuracy_fcp_position["Total passes"]
accuracy_fcp_position["% Total passes completed left"] = (accuracy_fcp_position["Passes completed left"]) * 100 / accuracy_fcp_position["Total passes left"]
accuracy_fcp_position["% Total passes completed right"] = (accuracy_fcp_position["Passes completed right"]) * 100 / accuracy_fcp_position["Total passes right"]
accuracy_fcp_position

,player_id,Passes completed left,Passes uncompleted left,Passes completed right,Passes uncompleted right,Total passes,Total passes completed,% Total passes completed,Total passes left,% Total passes completed left,Total passes right,% Total passes completed right
position,,,,,,,,,,,,
Midfielder,18693736,102,63,150,87,402,252,62.686567,165,61.818182,237,63.291139
Defender,24607177,54,32,71,45,202,125,61.881188,86,62.790698,116,61.206897
Forward,22380956,62,32,60,22,176,122,69.318182,94,65.957447,82,73.170732
Goalkeeper,778583,3,6,6,11,26,9,34.615385,9,33.333333,17,35.294118


In [15]:
allmatches["team_name"]=allmatches["team_name"].astype(str)

# % passing accuracy per zones by Benfica players in the league

accuracy_benfica = allmatches[allmatches["team_name"]=="Benfica"].groupby("player_name").sum()
# recalculate % completed passes
accuracy_benfica["% Total passes completed"] = ((accuracy_benfica["Passes completed left"]+accuracy_benfica["Passes completed right"]) * 100 / accuracy_benfica["Total passes"]).sort_values()
accuracy_benfica["% Total passes completed left"] = (accuracy_benfica["Passes completed left"]) * 100 / accuracy_benfica["Total passes left"]
accuracy_benfica["% Total passes completed right"] = (accuracy_benfica["Passes completed right"]) * 100 / accuracy_benfica["Total passes right"]
accuracy_benfica.sort_values("Total passes completed", ascending=False).fillna(0).drop(["player_id"], axis = 1)

,Passes completed left,Passes uncompleted left,Passes completed right,Passes uncompleted right,Total passes,Total passes completed,% Total passes completed,Total passes left,% Total passes completed left,Total passes right,% Total passes completed right
player_name,,,,,,,,,,,
Rafa,20,6,60,15,101,80,79.207921,26,76.923077,75,80.000000
Adel Taarabt,29,5,41,23,98,70,71.428571,34,85.294118,64,64.062500
Pizzi,29,8,39,9,85,68,80.000000,37,78.378378,48,81.250000
Éverton,45,10,12,4,71,57,80.281690,55,81.818182,16,75.000000
Diogo Gonçalves,0,0,48,7,55,48,87.272727,0,0.000000,55,87.272727
Alejandro Grimaldo,48,16,0,2,66,48,72.727273,64,75.000000,2,0.000000
Gian-Luca Waldschmidt,17,7,23,11,58,40,68.965517,24,70.833333,34,67.647059
Julian Weigl,20,4,19,8,51,39,76.470588,24,83.333333,27,70.370370
Gilberto,0,1,36,10,47,36,76.595745,1,0.000000,46,78.260870


In [13]:
allmatches["team_name"]=allmatches["team_name"].astype(str)

# % passing accuracy per zones by FCP players in the league

accuracy_fcp = allmatches[allmatches["team_name"]=="FC Porto"].groupby("player_name").sum()
# recalculate % completed passes
accuracy_fcp["% Total passes completed"] = ((accuracy_fcp["Passes completed left"]+accuracy_fcp["Passes completed right"]) * 100 / accuracy_fcp["Total passes"]).sort_values()
accuracy_fcp["% Total passes completed left"] = (accuracy_fcp["Passes completed left"]) * 100 / accuracy_fcp["Total passes left"]
accuracy_fcp["% Total passes completed right"] = (accuracy_fcp["Passes completed right"]) * 100 / accuracy_fcp["Total passes right"]
accuracy_fcp.sort_values("Total passes completed", ascending=False).fillna(0).drop(["player_id"], axis = 1)

,Passes completed left,Passes uncompleted left,Passes completed right,Passes uncompleted right,Total passes,Total passes completed,% Total passes completed,Total passes left,% Total passes completed left,Total passes right,% Total passes completed right
player_name,,,,,,,,,,,
Otávio,33,21,28,18,100,61,61.000000,54,61.111111,46,60.869565
Jesús Corona,12,8,43,23,86,55,63.953488,20,60.000000,66,65.151515
Wilson Manafá,23,5,29,13,70,52,74.285714,28,82.142857,42,69.047619
Mateus Uribe,21,11,25,10,67,46,68.656716,32,65.625000,35,71.428571
Sérgio Oliveira,20,16,24,23,83,44,53.012048,36,55.555556,47,51.063830
Mehdi Taremi,21,15,14,7,57,35,61.403509,36,58.333333,21,66.666667
Luis Díaz,27,5,8,1,41,35,85.365854,32,84.375000,9,88.888889
Zaidu Sanusi,23,13,4,2,42,27,64.285714,36,63.888889,6,66.666667
Chancel Mbemba,0,2,16,13,31,16,51.612903,2,0.000000,29,55.172414
